# 1) Write a python program to display IMDB’s Top rated 100 Indian movies’ data
https://www.imdb.com/list/ls056092300/ (i.e. name, rating, year ofrelease) and make data frame.


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.imdb.com/list/ls056092300/"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

movie_items = soup.find_all("div", class_="lister-item-content")

names = []
ratings = []
years = []

for item in movie_items:
    name = item.find("h3").find("a").text.strip()
    names.append(name)

    rating = item.find("span", class_="ipl-rating-star__rating").text.strip()
    ratings.append(rating)

    year = item.find("span", class_="lister-item-year").text.strip()
    years.append(year)

movie_data = pd.DataFrame({
    "Name": names,
    "Rating": ratings,
    "Year": years
})

print(movie_data)


                                 Name Rating    Year
0                     Ship of Theseus      8  (2012)
1                              Iruvar    8.4  (1997)
2                     Kaagaz Ke Phool    7.8  (1959)
3   Lagaan: Once Upon a Time in India    8.1  (2001)
4                     Pather Panchali    8.2  (1955)
..                                ...    ...     ...
95                        Apur Sansar    8.4  (1959)
96                        Kanchivaram    8.2  (2008)
97                    Monsoon Wedding    7.3  (2001)
98                              Black    8.1  (2005)
99                            Deewaar      8  (1975)

[100 rows x 3 columns]


# 4) Write a python program to scrape details of all the posts from https://www.patreon.com/coreyms .Scrape the
heading, date, content and the likes for the video from the link for the youtube video from the post.

In [26]:
import requests
from bs4 import BeautifulSoup

def scrape_patreon_posts(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    posts = []

    for post in soup.find_all('div', class_='post-card'):
        heading = post.find('h3', class_='post-card__title').text.strip()

        date = post.find('time', class_='post-card__published-at')['datetime']

        content = post.find('div', class_='post-card__excerpt').text.strip()

        youtube_link = post.find('a', href=lambda href: href and 'youtube.com' in href)
        likes = None
        if youtube_link:
            youtube_url = youtube_link['href']
            likes = get_youtube_likes(youtube_url)

        posts.append({
            'heading': heading,
            'date': date,
            'content': content,
            'likes': likes
        })

    return posts

def get_youtube_likes(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    likes_text = soup.find('button', attrs={'aria-label': 'like this video'})
    if likes_text:
        likes = likes_text.find_next('div').text.strip()
        return likes
    else:
        return None

def main():
    url = "https://www.patreon.com/coreyms"
    posts = scrape_patreon_posts(url)

    for post in posts:
        print("Heading:", post['heading'])
        print("Date:", post['date'])
        print("Content:", post['content'])
        print("Likes:", post['likes'])
        print()

if __name__ == "__main__":
    main()


# 5) Write a python program to scrape house details from mentioned URL. It should include house title, location,
area, EMI and price from https://www.nobroker.in/ .Enter three localities which are Indira Nagar, Jayanagar,
Rajaji Nagar.

In [25]:
import requests
from bs4 import BeautifulSoup

def scrape_house_details(locality):
    url = f"https://www.nobroker.in/property/sale/{locality}/bangalore?searchParam=W3sibGF0IjoxOC4wMDg3Mjg0LCJsb24iOjc3LjY4Mjg5MjQsInBsYWNlSWQiOiJDaElKbFpYM1hvY3pYM29zaVhzbVVaazZYT2wiLCJwbGFjZU5hbWUiOiJJbmRpcmEgTmFnYXIsIFJlYWQgQW5nZWwifV0=&radius=2.0"
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    houses = []

    house_cards = soup.find_all('div', class_='card')
    for card in house_cards:
        title = card.find('h2', class_='heading-6').text.strip()
        location = card.find('div', class_='nb__2CMjv').text.strip()
        area = card.find('div', class_='nb__3oNyC').text.strip()
        emi = card.find('div', class_='font-semi-bold heading-6').text.strip()
        price = card.find('div', class_='heading-7').text.strip()

        houses.append({
            'title': title,
            'location': location,
            'area': area,
            'emi': emi,
            'price': price
        })

    return houses

def main():
    localities = ["Indira%20Nagar", "Jayanagar", "Rajaji%20Nagar"]
    
    for locality in localities:
        print(f"Houses in {locality.replace('%20', ' ')}:")
        houses = scrape_house_details(locality)
        for house in houses:
            print(house)
        print()

if __name__ == "__main__":
    main()


Houses in Indira Nagar:

Houses in Jayanagar:

Houses in Rajaji Nagar:



# 6) Write a python program to scrape first 10 product details which include product name , price , Image URL from
https://www.bewakoof.com/bestseller?sort=popular . 

In [24]:
import requests
from bs4 import BeautifulSoup

def scrape_products(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    products = []

    product_cards = soup.find_all('div', class_='product-card')
    for card in product_cards[:10]:
        product_name_element = card.find('h4', class_='product-card__title')
        price_element = card.find('span', class_='product-card__price')
        image_element = card.find('img', class_='product-card__img')

        if product_name_element and price_element and image_element:
            product_name = product_name_element.text.strip()
            price = price_element.text.strip()
            image_url = image_element['src']

            products.append({
                'product_name': product_name,
                'price': price,
                'image_url': image_url
            })

    return products

def main():
    url = "https://www.bewakoof.com/bestseller?sort=popular"
    products = scrape_products(url)

    for index, product in enumerate(products, start=1):
        print(f"Product {index}:")
        print("Name:", product['product_name'])
        print("Price:", product['price'])
        print("Image URL:", product['image_url'])
        print()

if __name__ == "__main__":
    main()
